### Cargado de los archivos JSON

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import os

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        # get full path of the file
        #full_path = os.path.join(os.getcwd(), filename)
        #print("FULL PATH: ", full_path)
        
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None
    
def load_data_from_platform(platform : str, scenario : str):
    """Carga los datos de un escenario específico de una plataforma."""
    horarios_salas = os.path.join("..", platform + "_Output", scenario, "Horarios_salas.json")
    salas_input = os.path.join("..", "dataset", "scenarios", scenario, "salas.json")
    
    data = {
        'horarios_salas': load_json_file(horarios_salas),
        'salas_input': load_json_file(salas_input)
    }
    
    # print("CURRENT WORKING DIRECTORY: ", os.getcwd())

    if None in data.values():
        print("Error: No se pudieron cargar todos los archivos necesarios")
        return None
    return data

SCHEDULING_OUTPUT_DIR = "../scheduling_output"

# always assumed JADE and SPADE
def get_platform_output_dir(platform, metric_name, scenario_name):
    """Devuelve el directorio de salida para una plataforma dada."""
    full_path = os.path.join(SCHEDULING_OUTPUT_DIR, platform, metric_name, scenario_name)
    if not os.path.exists(full_path):
        os.makedirs(full_path)

    return full_path

# Compactación


In [2]:
def analyze_room_schedule_compact(sala):
    """Analiza el horario de una sala específica."""
    # Crear matriz de horario vacía (5 días x 9 bloques)
    schedule = [[False for _ in range(9)] for _ in range(5)]
    
    # Marcar bloques ocupados
    days = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
    for asignatura in sala['Asignaturas']:
        day_name = asignatura['Dia'].capitalize()
        day_idx = days[day_name]
        block_idx = asignatura['Bloque'] - 1
        schedule[day_idx][block_idx] = True
    
    # Contar ventanas
    ventanas_total = 0
    bloques_ocupados_total = 0
    
    for day in schedule:
        ventanas_dia = 0
        ocupados = 0
        in_window = False
        
        for i in range(9):
            if day[i]:  # Bloque ocupado
                ocupados += 1
                if in_window:
                    ventanas_dia += 1
                    in_window = False
            else:  # Bloque vacío
                if ocupados > 0:
                    in_window = True
        
        if in_window and ocupados > 0:
            ventanas_dia += 1
            
        ventanas_total += ventanas_dia
        bloques_ocupados_total += ocupados
    
    return {
        'ventanas': ventanas_total,
        'bloques_ocupados': bloques_ocupados_total,
        'schedule': schedule
    }

def calculate_window_durations(horarios_salas, platform, scenario):
    """Calcula la duración de las ventanas para cada sala."""
    try:
        window_stats = {}
        
        for sala in horarios_salas:
            codigo = sala['Codigo']
            ventanas = []
            
            # Crear matriz de ocupación
            schedule = [[False for _ in range(9)] for _ in range(5)]
            
            # Marcar bloques ocupados
            days = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
            for asignatura in sala['Asignaturas']:
                day_name = asignatura['Dia'].capitalize()
                day_idx = days[day_name]
                block_idx = asignatura['Bloque'] - 1
                schedule[day_idx][block_idx] = True
            
            # Encontrar ventanas y sus duraciones
            for day in schedule:
                window_size = 0
                prev_occupied = False
                for i in range(9):
                    if day[i]:  # Bloque ocupado
                        if window_size > 0:  # Si veníamos de una ventana
                            ventanas.append(window_size)
                            window_size = 0
                        prev_occupied = True
                    else:  # Bloque vacío
                        if prev_occupied:  # Solo contar ventanas entre bloques ocupados
                            window_size += 1
                if window_size > 0 and prev_occupied:  # Ventana al final del día
                    ventanas.append(window_size)
            
            # Agrupar ventanas por duración
            duration_counts = {}
            for duration in ventanas:
                if duration in duration_counts:
                    duration_counts[duration] += 1
                else:
                    duration_counts[duration] = 1
            
            window_stats[codigo] = {
                'duraciones': duration_counts,
                'total_bloques': sum(d * c for d, c in duration_counts.items())
            }
        
        return window_stats
        
    except Exception as e:
        print(f"Error al calcular las duraciones de ventanas: {str(e)}")
        return None

def amount_of_windows_lateral_chart(df_stats, platform, scenario):
    """Crea un gráfico de barras horizontal para la cantidad de ventanas por sala."""
    try:
        plt.figure(figsize=(12, 8))
        
        # Ordenar el DataFrame por cantidad de ventanas de forma descendente
        df_sorted = df_stats.sort_values('ventanas', ascending=True)
        
        # Crear gráfico de barras horizontal
        ax = plt.gca()
        bars = ax.barh(df_sorted.index, df_sorted['ventanas'], color='#00629B')
        
        # Configurar el aspecto del gráfico
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        plt.title('Cantidad de Ventanas por Sala')
        plt.xlabel('Número de Ventanas')
        
        # Añadir etiquetas de valor al final de cada barra
        for i, bar in enumerate(bars):
            width = bar.get_width()
            ax.text(width, bar.get_y() + bar.get_height()/2, 
                   f' {int(width)}',
                   va='center')
            
        full_path = get_platform_output_dir(platform, 'compactness', scenario)
        png_path = os.path.join(full_path, 'ventanas_cantidad.png')
        
        plt.tight_layout()
        plt.savefig(png_path)
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de cantidad de ventanas: {str(e)}")
        return False

def duration_of_windows_lateral_chart(window_stats, platform, scenario):
    """Crea un gráfico de barras compuestas para las duraciones de ventanas."""
    try:
        # Colores IEEE (cíclicos)
        colors = ['#99C2E8', '#00629B', '#A9A9A9']
        
        durations = [max(stats['duraciones'].keys()) 
                   for stats in window_stats.values() 
                   if stats['duraciones']]
        
        # Si no hay duraciones, retornar None
        if not durations:
            print("No hay datos de duración disponibles para crear el gráfico")
            return None
            
        max_duration = max(durations)
        
        # Preparar datos para el gráfico
        data = []
        
        for sala, stats in window_stats.items():
            duraciones = stats['duraciones']
            total_bloques = stats['total_bloques']
            
            # Crear lista ordenada de duraciones
            dur_list = []
            for d in range(1, max_duration + 1):
                if d in duraciones:
                    dur_list.append((d, duraciones[d]))
            
            data.append({
                'sala': sala,
                'duraciones': dur_list,
                'total_bloques': total_bloques
            })
        
        # Ordenar por total de bloques
        data.sort(key=lambda x: x['total_bloques'])
        
        # Crear gráfico
        fig, ax = plt.subplots(figsize=(15, 8))
        
        # Variables para posicionamiento
        y_pos = np.arange(len(data))
        left = np.zeros(len(data))
        
        # Crear barras compuestas
        for duration in range(1, max_duration + 1):
            widths = []
            labels = []
            for i, item in enumerate(data):
                width = 0
                for dur, count in item['duraciones']:
                    if dur == duration:
                        width = count * duration
                        break
                widths.append(width)
                
                # Crear etiqueta si hay valor
                if width > 0:
                    labels.append(f'{width}h')
                else:
                    labels.append('')
            
            # Dibujar barra
            color = colors[(duration - 1) % len(colors)]
            bars = ax.barh(y_pos, widths, left=left, 
                         color=color, label=f'{duration}h')
            
            # Añadir etiquetas
            for idx, (width, label) in enumerate(zip(widths, labels)):
                if width > 0:
                    x = left[idx] + width/2
                    text_color = 'white' if color == '#00629B' else 'black'
                    ax.text(x, y_pos[idx], label,
                           ha='center', va='center', color=text_color)
            
            left += widths
        
        # Configurar aspecto
        ax.set_yticks(y_pos)
        ax.set_yticklabels([item['sala'] for item in data])
        
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        plt.title('Duración de Ventanas por Sala')
        plt.xlabel('Total de Bloques en Ventanas')
        
        # Añadir total al final de cada barra
        for idx, item in enumerate(data):
            if item['total_bloques'] > 0:
                ax.text(left[idx] + 0.5, y_pos[idx],
                       f'{item["total_bloques"]}h',
                       va='center')
        
        # Leyenda
        plt.legend(title='Duración de Ventana', 
                  bbox_to_anchor=(1.1, 1),  # Ajustar más a la izquierda
                  loc='upper left')
        
        full_path = get_platform_output_dir(platform, 'compactness', scenario)
        png_path = os.path.join(full_path, 'ventanas_duracion.png')
        plt.tight_layout()
        plt.savefig(png_path, 
                   bbox_inches='tight')
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de duración de ventanas: {str(e)}")
        return False

def update_metrics_json(window_stats, df_stats, platform : str, scenario : str):
    """Actualiza el archivo de métricas con toda la información."""
    try:
        metrics = {
            'estadisticas_por_sala': df_stats.to_dict(orient='index'),
            'estadisticas_duracion': window_stats
        }
        
        path = get_platform_output_dir(platform, 'compactness', scenario)
        json_path = os.path.join(path, 'metricas_compactacion.json')
        
        # Guardar archivo
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(metrics, f, ensure_ascii=False, indent=2)
        
        return True
        
    except Exception as e:
        print(f"Error al actualizar el archivo de métricas: {str(e)}")
        return False
    
def compactaction_with_platform(platform : str):
    print(f"Iniciando análisis de compactación para {platform}")
    for scenario in ['full', 'medium', 'small']:
        # horarios_salas = load_json_file(f'../{platform}_Output/{scenario}/Horarios_salas.json')
        horarios_salas = load_data_from_platform(platform, scenario)['horarios_salas']

        # 2. Analizar cada sala
        results = {}
        
        for sala in horarios_salas:
            stats = analyze_room_schedule_compact(sala)
            results[sala['Codigo']] = stats
            
        # 3. Crear DataFrame con resultados
        df_stats = pd.DataFrame.from_dict(results, orient='index')

        # 4. Calcular duraciones de ventanas
        window_stats = calculate_window_durations(horarios_salas, platform, scenario)
        
        # 5. Crear visualizaciones
        # 5.1 Gráfico de cantidad de ventanas
        if not amount_of_windows_lateral_chart(df_stats, platform, scenario):
            return
            
        # 5.2 Gráfico de duración de ventanas
        if not duration_of_windows_lateral_chart(window_stats, platform, scenario):
            return
        
        if update_metrics_json(window_stats, df_stats, platform, scenario):
            print(f"\nAnálisis de compactación completado exitosamente para {scenario}")
        else:
            print(f"Error al completar el análisis de compactación para {scenario}")
            
    print(f"Análisis de compactación finalizado para {platform}")

# CAPACIDAD

In [3]:
def create_room_capacity_dict(salas_input):
    """Crea un diccionario con las capacidades de las salas."""
    try:
        return {room['Codigo']: room['Capacidad'] for room in salas_input}
    except KeyError as e:
        print(f"Error: Falta la clave {e} en los datos de las salas")
        return None

def categorize_capacity(current_capacity):
    """Categoriza la capacidad según los rangos definidos."""
    if current_capacity < 1.0:
        return 'bajo'
    elif current_capacity > 1.00:
        return 'sobre'
    else:
        return 'exacta'

def analyze_room_assignments(horarios_salas, room_capacities):
    """Analiza las asignaciones de cada sala."""
    capacity_stats = defaultdict(lambda: {'bajo': 0, 'exacta': 0, 'sobre': 0, 'total': 0})
    
    try:
        for sala in horarios_salas:
            codigo = sala['Codigo']
            
            for asignatura in sala['Asignaturas']:
                current_capacity = asignatura['Capacidad']
                capacity_stats[codigo]['total'] += 1
                category = categorize_capacity(current_capacity)
                capacity_stats[codigo][category] += 1
                
        return capacity_stats
    except KeyError as e:
        print(f"Error: Falta la clave {e} en los datos de asignaciones")
        return None

def create_capacity_dataframe(capacity_stats, room_capacities):
    """Crea un DataFrame con las estadísticas de capacidad."""
    try:
        df_stats = pd.DataFrame.from_dict(capacity_stats, orient='index')
        df_stats['room_capacity'] = df_stats.index.map(room_capacities)
        
        # Calcular porcentajes
        for col in ['bajo', 'exacta', 'sobre']:
            df_stats[f'{col}_pct'] = (df_stats[col] / df_stats['total'] * 100).round(2)
        
        # Crear rangos de capacidad
        df_stats['capacity_range'] = pd.cut(
            df_stats['room_capacity'], 
            bins=range(0, 91, 10), 
            labels=[f'{i}-{i+9}' for i in range(0, 81, 10)]
        )
        
        return df_stats
    except Exception as e:
        print(f"Error al crear el DataFrame: {str(e)}")
        return None

def create_summary_table(df_stats, platform : str, scenario : str):
    """Crea una tabla resumen de las estadísticas de capacidad."""
    try:
        summary_table = pd.DataFrame({
            'Sala': df_stats.index,
            'Capacidad Total': df_stats['room_capacity'],
            'Bajo Capacidad (%)': df_stats['bajo_pct'],
            'Capacidad Exacta (%)': df_stats['exacta_pct'],
            'Sobre Capacidad (%)': df_stats['sobre_pct']
        })
        
        full_path = get_platform_output_dir(platform, 'capacity', scenario)
        csv_path = os.path.join(full_path, 'resumen_capacidad.csv')
        # Guardar tabla en CSV
        summary_table.to_csv(csv_path, index=False)
        
        # Mostrar tabla formateada
        print("\nResumen de Capacidad por Sala:")
        print(summary_table.to_string(index=False))
        
        return summary_table
    except Exception as e:
        print(f"Error al crear la tabla resumen: {str(e)}")
        return None

def create_mix_chart(df_stats, platform : str, scenario : str):
    """Crea un gráfico combinado: donut chart y lateral bar chart."""
    try:
        # Calcular totales para las categorías
        total_counts = df_stats[['bajo', 'exacta', 'sobre']].sum()
        total_blocks = total_counts.sum()
        
        # Configurar el subplot con 1 fila y 2 columnas
        fig = plt.figure(figsize=(15, 6))
        
        # Crear el donut chart (izquierda)
        ax1 = plt.subplot(121)
        wedges, texts, autotexts = ax1.pie(total_counts, 
                labels=['Asientos Disponibles', 'Asientos justos', 'Asientos Insuficientes'],
                colors=['#99C2E8', '#00629B', '#A9A9A9'],  # Colores IEEE
                autopct='%1.1f%%',
                pctdistance=0.75)  # Ajustar la distancia de los porcentajes

        # Ajustar el tamaño y la posición de los porcentajes
        for i, autotext in enumerate(autotexts):
            if i == 0:  # Asientos Disponibles
                autotext.set_color('black')
            elif i == 1:  # Asientos justos
                autotext.set_color('white')
            else:  # Asientos Insuficientes
                autotext.set_color('black')
            autotext.set_fontsize(12)  # Cambiar el tamaño de la fuente
        
        # Crear el agujero central del donut
        centre_circle = plt.Circle((0,0), 0.50, fc='white')
        ax1.add_artist(centre_circle)
        
        # Agregar texto en el centro (suma de Bajo y Exacta)
        suma_porcentual = ((total_counts['bajo'] + total_counts['exacta']) / total_blocks * 100)
        plt.text(0, 0, f'{suma_porcentual:.1f}%', 
                ha='center', va='center', fontsize=20)
        
        ax1.set_title('Disponibilidad de Asientos Frente a la Asignacion de Salas')
        
        # Crear el bar chart horizontal (derecha)
        ax2 = plt.subplot(122)
        categories = ['Salas con Asientos Disponibles', 'Salas con Asientos Justos', 'Salas con Asientos Insuficientes']
        colors = ['#99C2E8', '#00629B', '#A9A9A9']

        # Create paired lists and sort them
        zipped = list(zip(total_counts, categories, colors))
        sorted_pairs = sorted(zipped, reverse=True)  # Sort in descending order
        total_counts_sorted, categories_sorted, colors_sorted = zip(*sorted_pairs)
        y_pos = range(len(categories))

        # Eliminar el borde superior y derecho
        ax2.spines['top'].set_visible(False)
        ax2.spines['right'].set_visible(False)

        # Create horizontal bars with sorted data
        ax2.barh(y_pos, total_counts_sorted, color=colors_sorted)

        # Invert y-axis to show larger values on top
        ax2.invert_yaxis()

        # Update labels with sorted categories
        ax2.set_yticks(y_pos)
        ax2.set_yticklabels(categories_sorted)
        ax2.yaxis.set_tick_params(pad=10)  # Ajustar la posición de las etiquetas
        ax2.set_title(f'Distribución de Asignaturas por Disponibilidad de Asientos\n(Total: {int(total_blocks)} asignaturas)')

        # Add values at end of bars
        for i, v in enumerate(total_counts_sorted):
            ax2.text(v, i, f' {int(v)}', va='center')

        # Agregar una línea divisoria entre los gráficos
        fig.subplots_adjust(wspace=0.5)  # Ajustar el Asientos entre subplots
        
        full_path = get_platform_output_dir(platform, 'capacity', scenario)
        png_path = os.path.join(full_path, 'capacidad_mixta.png')
        
        plt.tight_layout()
        plt.savefig(png_path)
        plt.close()
        
        return total_counts
        
    except Exception as e:
        print(f"Error al crear el gráfico mixto: {str(e)}")
        return None

def create_descriptive_statistics(summary_table, platform : str, scenario : str):
    try:
        # Estadísticas para Capacidad Total
        capacidad_stats = {
            'Métrica': [
                'Número de Salas',
                'Capacidad Mínima',
                'Primer Cuartil (Q1)',
                'Mediana',
                'Tercer Cuartil (Q3)',
                'Capacidad Máxima',
                'Rango Intercuartílico'
            ],
            'Valor': [
                len(summary_table),
                summary_table['Capacidad Total'].min(),
                summary_table['Capacidad Total'].quantile(0.25),
                summary_table['Capacidad Total'].median(),
                summary_table['Capacidad Total'].quantile(0.75),
                summary_table['Capacidad Total'].max(),
                summary_table['Capacidad Total'].quantile(0.75) - summary_table['Capacidad Total'].quantile(0.25)
            ]
        }
        
        # Análisis de distribución de capacidad
        ranges = [(0,20), (21,40), (41,60), (61,80), (81,100)]
        capacity_dist = []
        for start, end in ranges:
            count = len(summary_table[
                (summary_table['Capacidad Total'] >= start) & 
                (summary_table['Capacidad Total'] <= end)
            ])
            capacity_dist.append(f"Salas con {start}-{end} asientos: {count}")
        
        # Crear DataFrames
        df_stats = pd.DataFrame(capacidad_stats)
        
        full_path = get_platform_output_dir(platform, 'capacity', scenario)
        csv_path = os.path.join(full_path, 'descripcion_salas.csv')
        
        # Guardar en CSV
        df_stats.to_csv(csv_path, index=False)
        
        # Mostrar resultados
        print("\nDescripcion de Capacidad:")
        print(df_stats.to_string(index=False))
        print("\nDistribución de Capacidad:")
        for dist in capacity_dist:
            print(dist)
            
        return df_stats
        
    except Exception as e:
        print(f"Error al crear descripcion: {str(e)}")
        return None

def save_results_capacity(df_stats, total_counts, range_analysis, platform : str, scenario : str):
    """Guarda los resultados en un archivo JSON."""
    try:
        results = {
            'room_stats': df_stats.to_dict(orient='index'),
            'global_stats': {
                'bajo_capacidad': float(total_counts['bajo'] / total_counts.sum() * 100),
                'capacidad_exacta': float(total_counts['exacta'] / total_counts.sum() * 100),
                'sobre_capacidad': float(total_counts['sobre'] / total_counts.sum() * 100)
            },
            'range_analysis': range_analysis.to_dict()
        }
        
        json_path = get_platform_output_dir(platform, 'capacity', scenario)
        json_model_path = os.path.join(json_path, 'metricas_capacidad.json')
        
        with open(json_model_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        return True
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False
        
def capacity_with_platform(platform : str):
    print(f"Análisis de capacidad para {platform}")
    for scenario in ['full', 'medium', 'small']:
        print(f"Análisis de capacidad para {scenario}")
        data = load_data_from_platform(platform, scenario)
        if data is None:
            print(f"Error al cargar datos para {scenario}")
            continue
        
        # 2. Crear diccionario de capacidades
        room_capacities = create_room_capacity_dict(data['salas_input'])
        if room_capacities is None:
            print(f"Error al crear diccionario de capacidades para {scenario}")
            continue
        
        # 3. Analizar asignaciones
        capacity_stats = analyze_room_assignments(data['horarios_salas'], room_capacities)
        if capacity_stats is None:
            print(f"Error al analizar asignaciones para {scenario}")
            continue
        
        # 4. Crear DataFrame
        df_stats = create_capacity_dataframe(capacity_stats, room_capacities)
        if df_stats is None:
            print(f"Error al crear DataFrame para {scenario}")
            continue
        
        # 5. Crear tabla resumen
        summary_table = create_summary_table(df_stats, platform, scenario)
        if summary_table is None:
            print(f"Error al crear tabla resumen para {scenario}")
            continue
        
        # 5.1 Crear estadísticas descriptivas
        descriptive_stats = create_descriptive_statistics(summary_table, platform, scenario)
        if descriptive_stats is None:
            print(f"Error al crear estadísticas descriptivas para {scenario}")
            continue
        
        range_analysis = create_mix_chart(df_stats, platform, scenario)
        if range_analysis is None:
            print(f"Error al crear gráfico mixto para {scenario}")
            continue
        
        # 7. Guardar resultados
        if save_results_capacity(df_stats, range_analysis, descriptive_stats, platform, scenario):
            print(f"\nAnálisis de capacidad completado exitosamente para {scenario}")
        else:
            print(f"Error al completar el análisis de capacidad para {scenario}")
            
        print("="*100)

# TE (Time Eligibility)

In [4]:
def calculate_te(horarios_salas):
    """Calcula la métrica Time-slot Eligibility (TE)."""
    # Definir constantes
    DIAS = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
    TOTAL_PERIODOS = 9  # 9 bloques por día
    TOTAL_SLOTS = len(DIAS) * TOTAL_PERIODOS

    # Crear diccionario para almacenar eventos y sus períodos disponibles
    eventos = defaultdict(set)
    
    # Para cada sala, obtener los eventos y sus períodos ocupados
    for sala in horarios_salas:
        for asignatura in sala['Asignaturas']:
            evento_key = f"{asignatura['Nombre']}_{sala['Codigo']}"
            dia_name = asignatura['Dia'].capitalize()
            dia_idx = DIAS.index(dia_name)
            bloque = asignatura['Bloque']
            periodo = dia_idx * TOTAL_PERIODOS + bloque
            eventos[evento_key].add(periodo)

    # Si no hay eventos, retornar 0
    if not eventos:
        return 0

    # Calcular TE para cada evento
    te_values = []
    for evento, periodos_ocupados in eventos.items():
        # Los períodos disponibles son todos menos los ocupados
        periodos_disponibles = TOTAL_SLOTS - len(periodos_ocupados)
        te_individual = periodos_disponibles / TOTAL_SLOTS
        te_values.append(te_individual)

    # Calcular TE promedio
    te_promedio = sum(te_values) / len(eventos)
    
    return te_promedio
    
def TE_with_platform(platform : str):
    platform = platform.upper()
    print(f"Iniciando análisis de Time-slot Eligibility para {platform}")
    for scenario in ['full', 'medium', 'small']:
        # horarios_salas = load_json_file(f'{platform}_Output/{scenario}/Horarios_salas.json')
        horarios_salas = load_data_from_platform(platform, scenario)['horarios_salas']
        
        if not horarios_salas:
            print(f"Error al cargar datos para {scenario}")
            continue
        
        te = calculate_te(horarios_salas)
        print(f"TE promedio para {scenario}: {te:.4f}")
        
        # Guardar resultados
        results = {
            'scenario': scenario,
            'te_value': te,
            'te_percentage': te * 100
        }
        
        print(f"Resultados para {scenario}: {results}")
        
        full_path = get_platform_output_dir(platform, 'time_eligibility', scenario)
        json_path = os.path.join(full_path, 'metricas_te.json')
        
        # Guardar resultados en JSON
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        print(f"Resultados guardados en 'metricas_te.json' para {scenario}")

# OCUPACION

In [5]:
def create_schedule_matrix():
    """Crea una matriz de horario vacía (5 días x 9 bloques)."""
    return [[False for _ in range(9)] for _ in range(5)]

def analyze_room_schedule_occup(room_data):
    """Analiza el horario de una sala específica."""
    schedule = create_schedule_matrix()
    total_blocks = 45  # 9 bloques x 5 días
    
    # Verificar que tenemos los datos necesarios
    if not isinstance(room_data, dict) or 'Asignaturas' not in room_data:
        print(f"Error: Datos de sala inválidos: {room_data}")
        return None
        
    # Marcar bloques ocupados
    days = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
    for class_data in room_data.get('Asignaturas', []):
        day_name = class_data['Dia'].capitalize()
        day_idx = days[day_name]
        block_idx = class_data['Bloque'] - 1
        if 0 <= day_idx < 5 and 0 <= block_idx < 9:
            schedule[day_idx][block_idx] = True
    
    # Calcular estadísticas
    total_occupied = sum(row.count(True) for row in schedule)
    total_unoccupied = total_blocks - total_occupied
    
    return {
        'schedule': schedule,
        'total_occupied': total_occupied,
        'total_unoccupied': total_unoccupied
    }

def create_summary_dataframe(horarios_salas, platform : str, scenario : str):
    """Crea un DataFrame con el resumen de ocupación de todas las salas."""
    summary_data = []
    
    for sala in horarios_salas:
        if not isinstance(sala, dict) or 'Codigo' not in sala:
            print(f"Error: Estructura de sala inválida: {sala}")
            continue
            
        stats = analyze_room_schedule_occup(sala)
        if stats is None:
            continue
            
        summary_data.append({
            'Codigo': sala['Codigo'],
            'Bloques Ocupados': stats['total_occupied'],
            'Bloques Desocupados': stats['total_unoccupied'],
            'Tasa de Ocupacion': round(stats['total_occupied'] / (stats['total_occupied'] + stats['total_unoccupied']) * 100, 2)
        })
    
    df_summary = pd.DataFrame(summary_data)
    df_summary.set_index('Codigo', inplace=True)
    
    # Guardar tabla en CSV
    full_path = get_platform_output_dir(platform, 'occupancy', scenario)
    csv_path = os.path.join(full_path, 'resumen_ocupacion.csv')
    
    df_summary.to_csv(csv_path)
    
    return df_summary

def general_occupancy(df_stats, platform : str, scenario : str):
    """Crea un gráfico de torta mostrando la tasa de ocupación general."""
    try:
        plt.figure(figsize=(10, 6))
        
        # Calcular totales
        total_ocupados = df_stats['Bloques Ocupados'].sum()
        total_desocupados = df_stats['Bloques Desocupados'].sum()
        total = total_ocupados + total_desocupados
        
        # Calcular porcentajes
        porc_ocupados = (total_ocupados / total) * 100
        porc_desocupados = (total_desocupados / total) * 100
        
        # Crear gráfico de torta
        valores = [porc_ocupados, porc_desocupados]
        etiquetas = ['Bloques Ocupados', 'Bloques Desocupados']
        colores = ['#00629B', '#A9A9A9']  # Azul y gris
        
        patches, texts, autotexts = plt.pie(valores, 
                                        labels=etiquetas,
                                        colors=colores,
                                        autopct='%1.1f%%',
                                        startangle=90)
        
        # Ajustar color de texto
        for autotext in autotexts:
            if autotext.get_text().strip('%') == f'{porc_ocupados:.1f}':
                autotext.set_color('white')
            else:
                autotext.set_color('black')
        
        plt.title('Distribución General de Ocupación')
        plt.axis('equal')
        
        full_path = get_platform_output_dir(platform, 'occupancy', scenario)
        general_path = os.path.join(full_path, 'ocupacion_general.png')
        
        plt.savefig(general_path)
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de ocupación general: {str(e)}")
        return False

def room_occupancy(df_stats, platform : str, scenario : str):
    """Crea un gráfico de barras horizontal compuesto por sala."""
    try:
        # Ordenar salas por ocupación total descendente
        df_sorted = df_stats.sort_values('Bloques Ocupados', ascending=True)
        
        # Preparar datos
        salas = df_sorted.index
        ocupados = df_sorted['Bloques Ocupados']
        desocupados = df_sorted['Bloques Desocupados']
        y_pos = np.arange(len(salas))
        
        # Crear gráfico
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Barra de ocupados (azul)
        bars1 = ax.barh(y_pos, ocupados, color='#00629B', label='Bloques Ocupados')
        
        # Barra de desocupados (gris)
        bars2 = ax.barh(y_pos, desocupados, left=ocupados, 
                       color='#A9A9A9', label='Bloques Desocupados')
        
        # Configurar aspecto
        ax.set_yticks(y_pos)
        ax.set_yticklabels(salas)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        # Añadir etiquetas
        for bars, color_text in [(bars1, 'white'), (bars2, 'black')]:
            for bar in bars:
                width = bar.get_width()
                if width > 0:  # Solo mostrar etiqueta si hay valor
                    x = bar.get_x() + width/2
                    y = bar.get_y() + bar.get_height()/2
                    ax.text(x, y, f'{int(width)}', 
                           ha='center', va='center',
                           color=color_text)
        
        plt.title('Ocupación por Sala')
        plt.xlabel('Número de Bloques')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        
        full_path = get_platform_output_dir(platform, 'occupancy', scenario)
        salas_path = os.path.join(full_path, 'ocupacion_salas.png')
        
        plt.tight_layout()
        plt.savefig(salas_path, bbox_inches='tight')
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de ocupación por sala: {str(e)}")
        return False

def day_occupancy(horarios_salas, platform : str, scenario : str):
    """Crea un gráfico de barras vertical por día de la semana."""
    try:
        # Inicializar conteos por día
        dias = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
        ocupados_por_dia = {dia: 0 for dia in dias}
        total_por_dia = {dia: 0 for dia in dias}
        
        # Contar bloques ocupados por día
        for sala in horarios_salas:
            for asignatura in sala['Asignaturas']:
                dia = asignatura['Dia'].capitalize()
                ocupados_por_dia[dia] += 1
        
        # Calcular total de bloques posibles por día
        num_salas = len(horarios_salas)
        bloques_por_dia = 9  # 9 bloques por día
        for dia in dias:
            total_por_dia[dia] = num_salas * bloques_por_dia
        
        # Preparar datos para el gráfico
        ocupados = [ocupados_por_dia[dia] for dia in dias]
        desocupados = [total_por_dia[dia] - ocupados_por_dia[dia] for dia in dias]
        
        # Crear gráfico
        fig, ax = plt.subplots(figsize=(10, 6))
        x = np.arange(len(dias))
        width = 0.35
        
        # Barras ocupados (azul)
        bars1 = ax.bar(x, ocupados, width, label='Bloques Ocupados', color='#00629B')
        
        # Barras desocupados (gris)
        bars2 = ax.bar(x, desocupados, width, bottom=ocupados,
                      label='Bloques Desocupados', color='#A9A9A9')
        
        # Configurar aspecto
        ax.set_xticks(x)
        ax.set_xticklabels(dias)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        # Añadir etiquetas
        for bars, color_text in [(bars1, 'white'), (bars2, 'black')]:
            for bar in bars:
                height = bar.get_height()
                if height > 0:  # Solo mostrar etiqueta si hay valor
                    ax.text(bar.get_x() + bar.get_width()/2, 
                           bar.get_y() + height/2,
                           f'{int(height)}',
                           ha='center', va='center',
                           color=color_text)
        
        plt.title('Ocupación por Día')
        plt.ylabel('Número de Bloques')
        plt.legend()
        
        full_path = get_platform_output_dir(platform, 'occupancy', scenario)
        dias_path = os.path.join(full_path, 'ocupacion_dias.png')
        
        plt.tight_layout()
        plt.savefig(dias_path)
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de ocupación por día: {str(e)}")
        return False

def analyze_occupancy_quality(horarios_salas):
    """Analiza la calidad de ocupación por sala."""
    quality_stats = {}
    
    for sala in horarios_salas:
        codigo = sala['Codigo']
        quality_stats[codigo] = {
            'baja_utilizacion': 0,  # < 0.5
            'utilizacion_media': 0,  # 0.5 - 0.8
            'utilizacion_alta': 0,   # 0.8 - 1.0
            'sobre_utilizacion': 0,  # > 1.0
            'total_bloques': len(sala['Asignaturas']),  # Total de bloques asignados
            'uso_porcentual': (len(sala['Asignaturas']) / 45) * 100  # Porcentaje de uso total
        }
        
        for asignatura in sala['Asignaturas']:
            capacidad = asignatura['Capacidad']
            if capacidad < 0.5:
                quality_stats[codigo]['baja_utilizacion'] += 1
            elif 0.5 <= capacidad < 0.8:
                quality_stats[codigo]['utilizacion_media'] += 1
            elif 0.8 <= capacidad <= 1.0:
                quality_stats[codigo]['utilizacion_alta'] += 1
            else:
                quality_stats[codigo]['sobre_utilizacion'] += 1
    
    return quality_stats

def create_quality_chart(quality_stats, platform : str, scenario : str):
    """Crea un gráfico de barras apiladas para la calidad de ocupación."""
    try:
        # Convertir a DataFrame
        df = pd.DataFrame.from_dict(quality_stats, orient='index')
        
        # Calcular porcentajes relativos al total de bloques asignados
        categorias = ['baja_utilizacion', 'utilizacion_media', 'utilizacion_alta', 'sobre_utilizacion']
        for col in categorias:
            df[f'{col}_pct'] = (df[col] / df['total_bloques'] * 100).round(2)
        
        # Ordenar por uso porcentual (descendente)
        df_sorted = df.sort_values(by='uso_porcentual', ascending=True)
        
        # Crear gráfico
        fig, ax = plt.subplots(figsize=(15, 12))
        
        # Posición de las barras
        salas = df_sorted.index
        y_pos = np.arange(len(salas))
        
        # Colores IEEE
        colores = ['#99C2E8',  # Celeste
                  '#00629B',   # Azul
                  '#003F63',   # Azul oscuro
                  '#A9A9A9']   # Gris
        
        # Crear barras apiladas
        left = np.zeros(len(salas))
        bars_list = []
        
        # Primera barra completa (uso total)
        ax.barh(y_pos, df_sorted['uso_porcentual'], 
                color='#E6E6E6', alpha=0.3,  # Gris muy claro
                label='Capacidad total')
        
        # Barras apiladas de utilización
        for col, color in zip(['baja_utilizacion_pct', 'utilizacion_media_pct', 
                             'utilizacion_alta_pct', 'sobre_utilizacion_pct'], colores):
            # Ajustar el ancho según el uso porcentual
            width = df_sorted[col] * df_sorted['uso_porcentual'] / 100
            bars = ax.barh(y_pos, width, left=left, color=color)
            bars_list.append(bars)
            
            # Añadir etiquetas con número de bloques
            for i, v in enumerate(width):
                if v > 0:
                    x = left[i] + v/2
                    # Obtener el número real de bloques para esta categoría
                    num_bloques = df_sorted.iloc[i][col.replace('_pct', '')]
                    ax.text(x, y_pos[i], f'{int(num_bloques)}', 
                           ha='center', va='center',
                           color='white' if color in ['#00629B', '#003F63'] else 'black',
                           fontsize=8)
            left += width
        
        # Añadir etiquetas con el total de bloques asignados
        for i, (idx, row) in enumerate(df_sorted.iterrows()):
            ax.text(105, y_pos[i], 
                   f'Total: {int(row["total_bloques"])} bloques ({row["uso_porcentual"]:.1f}%)',
                   ha='left', va='center', fontsize=8)
        
        # Configurar aspecto
        ax.set_yticks(y_pos)
        ax.set_yticklabels(salas)
        ax.set_xlim(0, 120)  # Dar espacio para las etiquetas
        ax.invert_yaxis()  # Invertir eje Y para que las barras más altas estén arriba
        
        # Eliminar bordes
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        # Títulos y leyenda
        plt.title('Calidad de Ocupación por Sala', pad=20)
        plt.xlabel('Porcentaje de Uso')
        
        labels = ['Baja Utilización (<50%)', 
                 'Utilización Media (50-80%)',
                 'Utilización Alta (80-100%)',
                 'Sobre Utilización (>100%)']
        plt.legend(bars_list, labels, bbox_to_anchor=(1.05, 1), loc='upper left')
        
        full_path = get_platform_output_dir(platform, 'occupancy', scenario)
        calidad_path = os.path.join(full_path, 'calidad_ocupacion_grafico.png')
        
        plt.tight_layout()
        plt.savefig(calidad_path, bbox_inches='tight', dpi=300)
        plt.close()
        
        return df_sorted
        
    except Exception as e:
        print(f"Error al crear el gráfico de calidad de ocupación: {str(e)}")
        return None

def save_quality_results(quality_stats, df_quality : pd.DataFrame, platform : str, scenario : str):
    """Guarda los resultados de calidad de ocupación."""
    try:
        full_path = get_platform_output_dir(platform, 'occupancy', scenario)
        json_path = os.path.join(full_path, 'metricas_calidad_ocupacion.json')
        
        # Guardar JSON
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(quality_stats, f, ensure_ascii=False, indent=2)
            

        csv_path = os.path.join(full_path, 'calidad_ocupacion_tabla.csv')
        
        # Guardar CSV
        df_quality.to_csv(csv_path, encoding='utf-8')
        
        return True
    except Exception as e:
        print(f"Error al guardar los resultados de calidad: {str(e)}")
        return False

def calculate_daily_stats(horarios_salas):
    """Calcula estadísticas diarias de ocupación."""
    dias = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
    ocupados_por_dia = {dia: 0 for dia in dias}
    total_por_dia = {dia: 0 for dia in dias}
    
    num_salas = len(horarios_salas)
    bloques_por_dia = 9
    
    # Contar bloques ocupados por día
    for sala in horarios_salas:
        for asignatura in sala['Asignaturas']:
            dia = asignatura['Dia'].capitalize()
            ocupados_por_dia[dia] += 1
    
    # Calcular total y tasa de ocupación por día
    daily_stats = {}
    for dia in dias:
        total = num_salas * bloques_por_dia
        ocupados = ocupados_por_dia[dia]
        daily_stats[dia] = {
            'bloques_ocupados': ocupados,
            'bloques_desocupados': total - ocupados,
            'tasa_ocupacion': round(ocupados / total * 100, 2)
        }
    
    return daily_stats

def save_results_occupancy(df_stats, total_counts, daily_stats, platform : str, scenario : str):
    """Guarda los resultados en un archivo JSON."""
    try:
        results = {
            'room_stats': df_stats.to_dict(orient='index'),
            'global_stats': {
                'total_ocupados': int(total_counts[0]),
                'total_desocupados': int(total_counts[1]),
                'tasa_ocupacion_global': float(total_counts[0] / sum(total_counts) * 100)
            },
            'daily_stats': daily_stats
        }
        
        full_path = get_platform_output_dir(platform, 'occupancy', scenario)
        json_path = os.path.join(full_path, 'metricas_ocupacion.json')
        
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        return True
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False
        
def occupation_with_platform(platform : str):
    print(f"Iniciando análisis de ocupación para {platform}")
    for scenario in ['full', 'medium', 'small']:
        print(f"Procesando escenario: {scenario}")
        horarios_salas = load_json_file(f'../{platform}_Output/{scenario}/Horarios_salas.json')
        if horarios_salas is None:
            print(f"Error al cargar datos para {scenario}")
            continue
        
        # 2. Crear DataFrame con estadísticas y guardar CSV
        df_stats = create_summary_dataframe(horarios_salas, platform, scenario)
        if df_stats is None:
            print(f"Error al crear DataFrame para {scenario}")
            continue
        
        # 3. Calcular estadísticas diarias
        daily_stats = calculate_daily_stats(horarios_salas)
        
        # 4. Analizar calidad de ocupación
        quality_stats = analyze_occupancy_quality(horarios_salas)
        df_quality = create_quality_chart(quality_stats, platform, scenario)
        if df_quality is None:
            print(f"Error al crear DataFrame para {scenario}")
            continue
        
        if not save_quality_results(quality_stats, df_quality, platform, scenario):
            print(f"Error al guardar resultados de calidad para {scenario}")
            continue
        
        # 5. Crear los tres gráficos originales
        success = True
        
        # 5.1 Gráfico de ocupación general y obtener totales
        total_ocupados = df_stats['Bloques Ocupados'].sum()
        total_desocupados = df_stats['Bloques Desocupados'].sum()
        total_counts = [total_ocupados, total_desocupados]
        
        if not general_occupancy(df_stats, platform, scenario):
            success = False
        
        # 5.2 Gráfico de ocupación por sala
        if not room_occupancy(df_stats, platform, scenario):
            success = False
        
        # 5.3 Gráfico de ocupación por día
        if not day_occupancy(horarios_salas, platform, scenario):
            success = False
        
        # 6. Guardar resultados en JSON
        if not save_results_occupancy(df_stats, total_counts, daily_stats, platform, scenario):
            success = False
        
        if success:
            print(f"\nAnálisis de ocupación completado exitosamente para {scenario}")
        else:
            print(f"\nError al completar el análisis de ocupación para {scenario}")
            
        print("--------------------------------")

# ROOM ELIGIBILITY

In [6]:
from pathlib import Path

def calculate_re_metric(professors_data, rooms_data):
    """
    Calculate the RE (Salas Adecuadas por Evento) metric based on raw input data
    
    Args:
        professors_data: List of professor dictionaries with their courses
        rooms_data: List of classroom dictionaries
        
    Returns:
        float: The RE metric value (0-1)
    """
    # Extract all events (courses) from professors data
    all_events = []
    for professor in professors_data:
        for course in professor.get("Asignaturas", []):
            all_events.append({
                "CodigoAsignatura": course.get("CodigoAsignatura"),
                "Nombre": course.get("Nombre"),
                "Vacantes": course.get("Vacantes", 0),
                "Campus": course.get("Campus"),
                "Actividad": course.get("Actividad", "Teo"),
                "Nivel": course.get("Nivel", 0),
                "Profesor": professor.get("Nombre"),
                "TipoContrato": professor.get("TipoContrato", "JORNADA_COMPLETA")
            })
    
    # Total number of events and rooms
    total_events = len(all_events)
    total_rooms = len(rooms_data)
    
    if total_events == 0 or total_rooms == 0:
        return 0  # Avoid division by zero
    
    # Calculate suitable rooms for each event
    suitable_rooms_ratio_sum = 0
    event_suitable_counts = {}  # For detailed reporting
    
    for i, event in enumerate(all_events):
        suitable_rooms = 0
        
        for room in rooms_data:
            # Check if room is suitable using constraints
            is_suitable = is_room_suitable_simple(event, room)
            
            if is_suitable:
                suitable_rooms += 1
        
        # Store suitable room count for this event
        event_id = f"{event['CodigoAsignatura']} ({event['Nombre']})"
        event_suitable_counts[event_id] = suitable_rooms
        
        # Calculate the ratio of suitable rooms for this event
        event_ratio = suitable_rooms / total_rooms if total_rooms > 0 else 0
        suitable_rooms_ratio_sum += event_ratio
    
    # Calculate the average ratio across all events
    re_metric = suitable_rooms_ratio_sum / total_events if total_events > 0 else 0
    
    return re_metric, event_suitable_counts

def is_room_suitable_simple(event, room):
    """
    Simple version of room suitability check for RE metric calculation
    
    Args:
        event: Dictionary containing event details
        room: Dictionary containing room details
        
    Returns:
        bool: True if the room is suitable, False otherwise
    """
    # 1. Campus match check (hard constraint)
    if event.get("Campus") and room.get("Campus"):
        if event["Campus"] != room["Campus"]:
            return False
    
    # 2. Capacity check (hard constraint)
    if event.get("Vacantes") and room.get("Capacidad"):
        # Room must fit all students
        if event["Vacantes"] > room["Capacidad"]:
            return False
        
        # Small class check
        MEETING_ROOM_THRESHOLD = 10
        needs_small_room = event["Vacantes"] < MEETING_ROOM_THRESHOLD
        is_small_room = room["Capacidad"] < MEETING_ROOM_THRESHOLD
        
        if needs_small_room and not is_small_room:
            # Allow regular rooms if they're not extremely oversized
            if room["Capacidad"] > event["Vacantes"] * 4:
                return False
    
    # If passed all checks, the room is suitable
    return True

def analyze_schedule_quality(scheduled_profs_data, scheduled_rooms_data, original_profs_data, original_rooms_data):
    """
    Analyze the quality of the final schedule compared to initial constraints
    
    Args:
        scheduled_profs_data: Output JSON with professors' schedules
        scheduled_rooms_data: Output JSON with rooms' schedules
        original_profs_data: Input JSON with original professor data
        original_rooms_data: Input JSON with original room data
        
    Returns:
        dict: Schedule quality metrics
    """
    metrics = {
        "total_events": 0,
        "scheduled_events": 0,
        "campus_matches": 0,
        "good_capacity_fits": 0,
        "satisfaction_scores": []
    }
    
    # Analyze professor schedules
    for prof in scheduled_profs_data:
        for course in prof.get("Asignaturas", []):
            metrics["total_events"] += 1
            
            if "Sala" in course:  # This course was scheduled
                metrics["scheduled_events"] += 1
                
                # Find the scheduled room
                room_info = next((r for r in scheduled_rooms_data if r["Codigo"] == course["Sala"]), None)
                
                if room_info:
                    # Check campus match
                    if course.get("Campus") == room_info.get("Campus"):
                        metrics["campus_matches"] += 1
                    
                    # Check capacity fit
                    room_capacity = room_info.get("Capacidad", 0)
                    students = course.get("Vacantes", 0)
                    
                    if students > 0 and room_capacity > 0:
                        occupancy_ratio = students / room_capacity
                        if 0.75 <= occupancy_ratio <= 0.95:
                            metrics["good_capacity_fits"] += 1
                    
                    # Get satisfaction score if available
                    if "Satisfaccion" in course:
                        metrics["satisfaction_scores"].append(course["Satisfaccion"])
    
    # Calculate ratios and averages
    if metrics["total_events"] > 0:
        metrics["scheduling_rate"] = metrics["scheduled_events"] / metrics["total_events"]
    else:
        metrics["scheduling_rate"] = 0
        
    if metrics["scheduled_events"] > 0:
        metrics["campus_match_rate"] = metrics["campus_matches"] / metrics["scheduled_events"]
        metrics["good_capacity_rate"] = metrics["good_capacity_fits"] / metrics["scheduled_events"]
    else:
        metrics["campus_match_rate"] = 0
        metrics["good_capacity_rate"] = 0
        
    if metrics["satisfaction_scores"]:
        metrics["avg_satisfaction"] = sum(metrics["satisfaction_scores"]) / len(metrics["satisfaction_scores"])
    else:
        metrics["avg_satisfaction"] = 0
    
    return metrics

def re_pre_analysis(platform: str):
    """
    Calculate RE metric and analyze schedules, saving results to JSON files
    
    Args:
        platform: Name of the platform being analyzed (JADE, SPADE, etc.)
        output_dir: Directory to save the results
    """
    print(f"Iniciando análisis de RE para {platform}")
    
    output_dir = get_platform_output_dir(platform, 'RE', "all")
    
    # Create output directory if it doesn't exist
    # Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Dictionary to store all results
    all_results = {
        "platform": platform,
        "scenarios": {}
    }
    
    for scenario in ['full', 'medium', 'small']:
        print(f"Processing scenario: {scenario}")
        
        scenario_results = {
            "re_metric": 0,
            "re_percentage": 0,
            "total_events": 0,
            "total_rooms": 0,
            "event_details": {},
            "limited_flexibility_events": [],
            "schedule_quality": None
        }
        
        try:
            rooms_path = os.path.join("..", "dataset", "scenarios", scenario, "salas.json")
            professors_path = os.path.join("..", "dataset", "scenarios", scenario, "profesores.json")
            
            # Read input data (pre-scheduling)
            with open(professors_path, 'r', encoding='utf-8') as f:
                professors_data = json.load(f)
            
            with open(rooms_path, 'r', encoding='utf-8') as f:
                rooms_data = json.load(f)
            
            # Calculate RE metric (pre-scheduling flexibility)
            re_metric, event_counts = calculate_re_metric(professors_data, rooms_data)
            
            # Save results to scenario dictionary
            scenario_results["re_metric"] = re_metric
            scenario_results["re_percentage"] = re_metric * 100  # Convert to percentage
            scenario_results["total_events"] = len(event_counts)
            scenario_results["total_rooms"] = len(rooms_data)
            scenario_results["event_details"] = event_counts
            
            print(f"RE Metric: {re_metric:.4f}")
            print(f"RE Percentage: {re_metric * 100:.2f}%")
            print(f"Total events: {len(event_counts)}")
            print(f"Total rooms: {len(rooms_data)}")
            
            # Find events with limited room options
            low_flexibility_threshold = 0.2
            limited_flexibility_events = []
            
            print("\nEvents with limited room options:")
            for event, count in event_counts.items():
                ratio = count / len(rooms_data)
                if ratio < low_flexibility_threshold:
                    event_info = {
                        "event": event,
                        "suitable_rooms": count,
                        "total_rooms": len(rooms_data),
                        "flexibility_ratio": ratio,
                        "flexibility_percentage": ratio * 100
                    }
                    limited_flexibility_events.append(event_info)
                    print(f"  {event}: {count}/{len(rooms_data)} suitable rooms ({ratio:.1%})")
            
            scenario_results["limited_flexibility_events"] = limited_flexibility_events
            
            # Optionally analyze the quality of the final schedule
            try:
                professors_output_file = os.path.join("..", f"{platform}_Output", scenario, "Horarios_asignados.json")
                rooms_output_file = os.path.join("..", f"{platform}_Output", scenario, "Horarios_salas.json")
                
                with open(professors_output_file, 'r', encoding='utf-8') as f:
                    professors_output = json.load(f)
                
                with open(rooms_output_file, 'r', encoding='utf-8') as f:
                    rooms_output = json.load(f)
                
                # Analyze schedule quality
                schedule_metrics = analyze_schedule_quality(
                    professors_output, rooms_output,
                    professors_data, rooms_data
                )
                
                # Convert scheduling rate and other metrics to percentages
                for key in ["scheduling_rate", "campus_match_rate", "good_capacity_rate"]:
                    if key in schedule_metrics:
                        schedule_metrics[f"{key}_percentage"] = schedule_metrics[key] * 100
                
                scenario_results["schedule_quality"] = schedule_metrics
                
                print("\nSchedule Quality Metrics:")
                print(f"Scheduled events: {schedule_metrics['scheduled_events']}/{schedule_metrics['total_events']} ({schedule_metrics['scheduling_rate']:.1%})")
                print(f"Campus match rate: {schedule_metrics['campus_match_rate']:.1%}")
                print(f"Good capacity fit rate: {schedule_metrics['good_capacity_rate']:.1%}")
                print(f"Average satisfaction score: {schedule_metrics['avg_satisfaction']:.2f}/10")
                
            except (FileNotFoundError, json.JSONDecodeError) as e:
                print(f"\nNote: Could not analyze schedule quality: {str(e)}")
        
        except FileNotFoundError as e:
            print(f"Error: One of the input files was not found. {str(e)}")
            scenario_results["error"] = f"File not found: {str(e)}"
        except json.JSONDecodeError as e:
            print(f"Error: One of the input files contains invalid JSON. {str(e)}")
            scenario_results["error"] = f"Invalid JSON: {str(e)}"
        except Exception as e:
            print(f"Unexpected error: {str(e)}")
            scenario_results["error"] = f"Unexpected error: {str(e)}"
        
        # Add scenario results to the all_results dictionary
        all_results["scenarios"][scenario] = scenario_results
    
    # Save all results to a JSON file
    output_file = os.path.join(output_dir, f"{platform}_RE_analysis.json")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, indent=2)
    
    print(f"\nResults saved to {output_file}")
    return all_results

def compare_platforms_re(platforms, output_dir="RE_Results"):
    """
    Compare RE metrics across different platforms
    
    Args:
        platforms: List of platform names to compare
        output_dir: Directory where results are stored
    """
    comparison = {
        "platforms": {},
        "summary": {}
    }
    
    for platform in platforms:
        try:
            # Load results for this platform
            file_path = os.path.join(output_dir, f"{platform}_RE_analysis.json")
            with open(file_path, 'r', encoding='utf-8') as f:
                platform_results = json.load(f)
            
            # Extract RE metrics for each scenario
            platform_re = {}
            for scenario, results in platform_results["scenarios"].items():
                platform_re[scenario] = {
                    "re_metric": results.get("re_metric", 0),
                    "re_percentage": results.get("re_percentage", 0)
                }
            
            comparison["platforms"][platform] = platform_re
            
        except (FileNotFoundError, json.JSONDecodeError) as e:
            print(f"Error loading results for {platform}: {str(e)}")
    
    # Calculate overall averages and compile summary
    for scenario in ["full", "medium", "small"]:
        scenario_values = []
        for platform, scenarios in comparison["platforms"].items():
            if scenario in scenarios:
                scenario_values.append(scenarios[scenario]["re_metric"])
        
        # Calculate statistics for this scenario
        if scenario_values:
            comparison["summary"][scenario] = {
                "average_re": sum(scenario_values) / len(scenario_values),
                "average_re_percentage": (sum(scenario_values) / len(scenario_values)) * 100,
                "min_re": min(scenario_values),
                "min_re_percentage": min(scenario_values) * 100,
                "max_re": max(scenario_values),
                "max_re_percentage": max(scenario_values) * 100
            }
    
    # Save comparison to a JSON file
    output_file = os.path.join(output_dir, "platforms_RE_comparison.json")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(comparison, f, indent=2)
    
    print(f"Platform comparison saved to {output_file}")
    
    # Print comparison summary
    print("\nRE Metric Comparison Summary:")
    print("-----------------------------")
    for scenario, stats in comparison["summary"].items():
        print(f"\nScenario: {scenario}")
        print(f"Average RE: {stats['average_re']:.4f} ({stats['average_re_percentage']:.2f}%)")
        print(f"Min RE: {stats['min_re']:.4f} ({stats['min_re_percentage']:.2f}%)")
        print(f"Max RE: {stats['max_re']:.4f} ({stats['max_re_percentage']:.2f}%)")
    
    return comparison


# EJECUCION DE LA SOLUCION
FIXME: URGENTLY!!! FIND THE ROOM ELIGIBILITY METRIC AND FIX IT.

In [7]:
if __name__ == "__main__":
    platforms = ["JADE", "SPADE"]
    for platform in platforms:
        capacity_with_platform(platform)
        TE_with_platform(platform)
        occupation_with_platform(platform)
        compactaction_with_platform(platform)
        re_pre_analysis(platform)
        print(f"Finished analysis for {platform}.\n")

Análisis de capacidad para JADE
Análisis de capacidad para full

Resumen de Capacidad por Sala:
 Sala  Capacidad Total  Bajo Capacidad (%)  Capacidad Exacta (%)  Sobre Capacidad (%)
   LP               29              100.00                  0.00                  0.0
 V207               60               83.72                 16.28                  0.0
 V205               65               93.33                  6.67                  0.0
 LFIS               39              100.00                  0.00                  0.0
  CM3               50               44.19                 55.81                  0.0
 A106               18              100.00                  0.00                  0.0
  CM5               45               84.44                 15.56                  0.0
  CM4               45               95.56                  4.44                  0.0
   E1               45               93.33                  6.67                  0.0
  CM6               30                0.00  

C:\Users\fccod\AppData\Local\Temp\ipykernel_2432\3618098923.py:313: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = left[i] + v/2



Análisis de ocupación completado exitosamente para full
--------------------------------
Procesando escenario: medium


C:\Users\fccod\AppData\Local\Temp\ipykernel_2432\3618098923.py:313: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = left[i] + v/2



Análisis de ocupación completado exitosamente para medium
--------------------------------
Procesando escenario: small


...


Análisis de ocupación completado exitosamente para full
--------------------------------
Procesando escenario: medium


C:\Users\fccod\AppData\Local\Temp\ipykernel_2432\3618098923.py:313: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = left[i] + v/2



Análisis de ocupación completado exitosamente para medium
--------------------------------
Procesando escenario: small


C:\Users\fccod\AppData\Local\Temp\ipykernel_2432\3618098923.py:313: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = left[i] + v/2



Análisis de ocupación completado exitosamente para small
--------------------------------
Iniciando análisis de compactación para SPADE

Análisis de compactación completado exitosamente para full

Análisis de compactación completado exitosamente para medium

Análisis de compactación completado exitosamente para small
Análisis de compactación finalizado para SPADE
Iniciando análisis de RE para SPADE
Processing scenario: full
RE Metric: 0.4352
RE Percentage: 43.52%
Total events: 294
Total rooms: 73

Events with limited room options:
  (IC801-A) (DIRECCION DE PROYECTOS): 13/73 suitable rooms (17.8%)
  (ARQ23-A) (HISTORIA Y TEORIA I): 6/73 suitable rooms (8.2%)
  (ARQ22-A) (GEOMETRIA Y ARQUITECTURA): 6/73 suitable rooms (8.2%)
  (PRI62-A) (LEGISLACIÓN DE SALUD Y SE): 7/73 suitable rooms (9.6%)
  (PRI66-A) (TALLER DE INTEGRACIÓN PRO): 7/73 suitable rooms (9.6%)
  (FIA41-B) (TERMODINÁMICA): 7/73 suitable rooms (9.6%)
  (IC804-A) (GESTIÓN DE PRODUCCIÓN): 13/73 suitable rooms (17.8%)
  (IC904